# Objective: To develop a mathematical understanding of PCA  
Principal Components Analysis, also known as PCA, is a technique commonly used for reducing the dimensionality of data while preserving as much as possible of the information contained in the original data. PCA achieves this goal by projecting data onto a lower-dimensional subspace that retains most of the variance among the data points.   
(https://programmathically.com/principal-components-analysis-explained-for-dummies/)  

Consider this simple two-dimensional graphical explanation of what we aim to do here:  
![SpaceRotationGif](../assets/capture_variance_pca.gif)  
This exercise attempts to convey mathematically how we achieve the transformation as we see in the animation. If you find any part of the exercise unclear, we highly recommend watching this StatQuest video to get a primer on PCA before solving the exercise: 
https://www.youtube.com/watch?v=FgakZw6K1QQ&ab_channel=StatQuestwithJoshStarmer

# Test data

In [ ]:
import numpy as np


X = np.array([
    [0.387, 4878, 5.42],
    [0.723, 12104, 5.25],
    [1, 12756, 5.52],
    [1.524, 6787, 3.94],
])
X -= np.mean(X, axis=0)

# Non-Iinear Iterative Partial Least-Squares (NIPALS) algorithm

Steps to compute PCA using NIPALS algorithm

* Step 1: Initialize an arbitrary column vector $\mathbf{t}_{a}$ either randomly or by just copying any column of X. 
* Step 2: Take very column of $\mathbf{X}$, $\mathbf{X_k}$ and regress it onto the $\mathbf{t}_{a}$ vector and store the regression coefficeints as $\mathbf{p}_{ka}$. (Note: This simply means performing an ordinary least squares regression ($y=mx$) with $x=t_{a}$ and $y=X_{k}$ with $m=(\mathbf{x^\top}\mathbf{x})^{-1}\mathbf{x^\top}\mathbf{y}$). In the current notation we get 
$$p_{ka}=\frac{\mathbf{t_a^\top}\mathbf{X}_{k}}{\mathbf{t_a^\top}\mathbf{t_a}}$$

Repeat it for each of the columns of $X$ to get the entire vector $\mathbf{p}_{k}$. This is shown in the illustration
above where each column from $X$ is regressed, one at a time, on $\mathbf{t}_{a}$, to calculate the loading entry, $\mathbf{𝑝}_{ka}$ 

In practice we don’t do this one column at time; we can regress all columns in $\mathbf{X}$ in go: $$\mathbf{p_a^\top}=\frac{1}{\mathbf{t_a^\top}\mathbf{t_a}}.\mathbf{t_a^\top}\mathbf{X_a}$$  where $\mathbf{t_a}$ is an $N \times 1$ column vector, and $\mathbf{X}_{a}$ us an $N \times K$ matrix.
* The loading vector $\mathbf{p_a^\top}$ won’t have unit length (magnitude) yet. So we simply rescale it to have
magnitude of 1.0: $$\mathbf{p_a^\top}=\frac{\mathbf{p_a^\top}}{\sqrt{\mathbf{p_a^\top}\mathbf{p_a}}}$$
* Step 4: Regress every row in $\mathbf{X}$ onto this normalized loadings vector. As illustrated below, in our linear regression the rows in $\mathbf{X}$ are our y-variable each time, while the loadings vector is our x-variable. The regression coefficient becomes the score value for that $𝑖^{th}$ row:

$$p_{i,a}=\frac{\mathbf{x}_{i}^{\top}\mathbf{p}_{a}}{\mathbf{p}_{a}^{\top}\mathbf{p}_{a}}$$
where $x_{i}^{T}$ is a $K \times 1$ column vector. We can combine these $N$ separate least-squares models and
calculate them in one go to get the entire vector, 

$$\mathbf{t}_{a}^{\top}=\frac{1}{\mathbf{p}_{a}^{\top}\mathbf{p}_{a}}\mathbf{X}\mathbf{p}_{a}^{\top}$$  where $p_{a}$ is a $K \times 1$ column vector.
* Step 5: Continue looping over steps 2,3,4 until the change in vector $t_{a}$ is below a chosen tolerance
* Step 6: On convergence, the score vector and the loading vectors, $\mathbf{t}_{a}$ and $\mathbf{p}_{a}$ are stored as the $a^{th}$ column in matrix $\mathbf{T}$ and $\mathbf{P}$. We then deflate the $\mathbf{X}$ matrix. This crucial step removes the variability captured in this component ($t_{a}$ and $p_{a}$) from $\mathbf{X}$:

$$E_{a}=X_{a}-t_{a}p_{a}^{\top}$$

$$X_{a+1} = E_{a}$$ 

For the first component, $X_{a}$ is just the preprocessed raw data. So we can see that the second component is actually calculated on the residuals $E_{1}$, obtained after extracting the first component. This is called deflation, and nicely shows why each component is orthogonal to the others. Each subsequent component is only seeing variation remaining after removing all the others; there is no possibility that two components can explain the same type of variability. After deflation we go back to step 1 and repeat the entire process for the next component. 

## Implementation in Python

In [ ]:
sum_variance = lambda X: np.sum(np.var(X, axis=0, ddof=1))


def PCA(X, n_components, tol=1e-7, max_iter=1000):
    """
    Compute PCA using the NIPALS algorithm.
    
    Args:
        X (np.ndarray): Training data.
        n_components (int): Number of components to keep.
        tol (float): Convergence tolerance.
        max_iter (int): Maximum iterations before stopping.
    Returns:
        T (np.ndarray): Scores matrix.
        P (np.ndarray): Loadings matrix.
        var_explained (np.ndarray): Fraction of variance explained by each PC.
    """
    # Initialize variables
    n_obs, n_vars = X.shape
    T = np.zeros((n_obs, n_components))
    P = np.zeros((n_vars, n_components))
    var_explained = np.zeros(n_vars)
    
    # Center data and calculate total variance
    X_a = X - np.mean(X, axis=0)
    total_var = sum_variance(X_a)
    current_var = total_var
    
    for h in range(n_components):
        # Choose t_h as any column of X_h
        t_h = X_a[:, 0].reshape(-1, 1)
        n_r = 0
        
        finished = False
        while not finished:
            n_r += 1
            # Compute loadings
            p_h = np.dot(X_a.T, t_h) / np.dot(t_h.T, t_h)
            # Normalize loadings
            p_h /= np.linalg.norm(p_h)
            # Compute scores
            t_h_new = np.dot(X_a, p_h) / np.dot(p_h.T, p_h)
            
            error = (t_h_new - t_h).T @ (t_h_new - t_h)
            t_h = t_h_new
            
            if error <= tol ** 2:
                finished = True
            elif n_r >= max_iter:
                print("Iteration stopped without convergence.")
                finished = True

        # Deflate X_a
        X_a -= t_h @ p_h.T
        
        T[:, h] = t_h.flatten()
        P[:, h] = p_h.flatten()
        
        old_var = current_var
        current_var = sum_variance(X_a)
        var_explained[h] = (old_var - current_var) / total_var
        
    return T, P, var_explained

### Advantages of the NIPALS algorithm
* The NIPALS algorithm computes one component at a time. The first component computed is
equivalent to the t1 and p1 vectors that would have been found from an eigenvalue or singular value
decomposition.
* The algorithm can handle missing data in X.
* The algorithm always converges, but the convergence can sometimes be slow.
* It is also known as the Power algorithm to calculate eigenvectors and eigenvalues.
* It works well for very large data sets.
* It is used by most software packages, especially those that handle missing data.
* Of interest: it is well known that Google used this algorithm for the early versions of their search engine, called PageRank148.

In [ ]:
n_components = 3
T, P, pcvar = PCA(X, n_components, max_iter=1e3)

print("T (Scores)", T, sep="\n", end="\n\n")
print("P (Loadings)", P, sep="\n", end="\n\n")
print("Variance explained ratio", np.sqrt(pcvar) / np.sum(np.sqrt(pcvar)), sep="\n")

## SVD implementation

In [ ]:
from numpy.linalg import svd


U, S, P_T = svd(X, full_matrices=False)
Sigma = np.diag(S)
T = np.dot(U,Sigma)
P = P_T.T

print("T (Scores)", T, sep="\n", end="\n\n")
print("P (Loadings)", P, sep="\n", end="\n\n")
print("Sigma (Variance)", S, sep="\n")

## Sklearn PCA implementation

In [ ]:
from sklearn.decomposition import PCA


pca = PCA()

T = pca.fit_transform(X)
P = pca.components_.T
latent = pca.explained_variance_
explained = pca.explained_variance_ratio_
S = pca.singular_values_
Sigma = np.diag(S)

print("T (Scores)", T, sep="\n", end="\n\n")
print("P (Loadings)", P, sep="\n", end="\n\n")
print("Sigma (Variance)", S, sep="\n")

#### Observe that we can compute the explained variance ratios using the singular values returned from the PCA calculation

In [ ]:
pca.explained_variance_ratio_

In [ ]:
explained_variance_2 = (S ** 2) / 4
explained_variance_ratio_2 = (explained_variance_2 / explained_variance_2.sum())
print(explained_variance_ratio_2)

## Eigenvalue decomposition PCA implementation

Recall that the latent variable directions (the loading vectors) were oriented so that the variance of the
scores in that direction were maximal. We can cast this as an optimization problem. For the first
component:  
$$max (\phi)=\mathbf{t_1^\top}\mathbf{t_1}=\mathbf{p_1^\top} \mathbf{X^\top}\mathbf{Xp_1}$$
such that $\mathbf{p_1^\top p_1}=1$.

This is equivalent to $$max (\phi)=\mathbf{p_1^\top} \mathbf{X^\top Xp_1}-\lambda(\mathbf{p_1^\top}\mathbf{p_1}-1)$$ 

because we can move the constraint into the objective function with a Lagrange multiplier, $\lambda$. The maximum value must occur when the partial derivatives with respect to $\mathbf{p_1}$, 

our search variable, are zero: $$\frac{\partial \phi}{\partial p_1}= \frac{\partial (\mathbf{p_1^\top X^\top Xp_1}-\lambda(\mathbf{p}_{1}^{\top}\mathbf{p}_{1}-1))}{\partial \mathbf{p}_1}=0$$

$$2\mathbf{X^\top X p_1}-2\lambda_1\mathbf{p_1}=0$$

$$(\mathbf{X^\top X}-\lambda_1\mathbf{I})\mathbf{p_1}=0$$

$$\mathbf{X^\top Xp_1}=\lambda_{1}\mathbf{p_1}$$

which is just the eigenvalue equation, indicating that $\mathbf{p_1}$ is the eigenvector of $\mathbf{X^\top X}$ and $\lambda_1$ is the eigenvalue. One can show that $\lambda_1=\mathbf{t_1^\top t_1}$, which is proportional to the variance of the first component. In a similar manner we can calculate the second eigenvalue, but this time we add the additional constraint that $\mathbf{p}_1 \perp \mathbf{p}_2$. Writing out this objective function and taking partial derivatives leads to showing that 

$$\mathbf{X^\top Xp_2} = \lambda_2 \mathbf{p_2}.$$

From this we learn that:
* The loadings are the eigenvectors of $\mathbf{X^\top X}$.
* Sorting the eigenvalues in order from largest to smallest gives the order of the corresponding eigenvectors, the loadings.
* We know from the theory of eigenvalues that if there are distinct eigenvalues, then their eigenvectors are linearly independent (orthogonal).
* We also know the eigenvalues of $\mathbf{X^\top X}$ must be real values and positive; this matches with the interpretation that the eigenvalues are proportional to the variance of each score vector.
* Also, the sum of the eigenvalues must add up to sum of the diagonal entries of $\mathbf{X^\top X}$, which represents of the total variance of the $\mathbf{X}$ matrix, if all eigenvectors are extracted. So plotting the eigenvalues is equivalent to showing the proportion of variance explained in X by each component. This is not necessarily a good way to judge the number of components to use, but it is a rough guide: use a Pareto plot of the eigenvalues (though in the context of eigenvalue problems, this plot is called a scree plot).

In [ ]:
from scipy import linalg as la


cov = np.cov(X, rowvar=False)
evals, P = la.eigh(cov)
idx = np.argsort(evals)[::-1]
P = P[:, idx]
evals = evals[idx]
T = np.dot(X, P) 
Sigma = la.norm(T, axis=0)

print("T (Scores)", T, sep="\n", end="\n\n")
print("P (Loadings)", P, sep="\n", end="\n\n")
print("Sigma (Variance)", S, sep="\n")

## Task 1: Test if the loading vectors are orthogonal and orthonormal or not  
Implement the functions `is_orthogonal` and `is_orthonormal`, which both return a boolean value.  
*Hint*: We can tolerate a small amount of error. The functions `np.allclose` and `np.eye` can be helpful here.

In [ ]:
def is_orthogonal(matrix: np.ndarray) -> bool:
    """
    Check if the matrix is orthogonal.

    Returns:
        _ (bool): Whether or not the column vectors are orthogonal.
    """
    # TODO: Implement the function here
    
    
orthogonal_P = is_orthogonal(P)
assert isinstance(orthogonal_P, bool), "You need to implement the function"
result_string = "orthogonal" if orthogonal_P else "not orthogonal"
print(f"The loading vectors are {result_string}")

In [ ]:
def is_orthonormal(matrix: np.ndarray) -> bool:
    """
    Check if the matrix is orthonormal.

    Returns:
        _ (bool): Whether or not the column vectors are orthonormal.
    """
    # TODO: Implement the function here
    # Hint: The function 'is_orthogonal' can be helpful here as well
    
    
orthonormal_P = is_orthonormal(P)
assert isinstance(orthonormal_P, bool), "You need to implement the function"
result_string = "orthonormal" if orthonormal_P else "not orthonormal"
print(f"The loading vectors are {result_string}")

## Task 2: Test if the scores vectors are orthogonal and orthonormal or not  
*Hint*: Code is made to be reused.

In [ ]:
# TODO: Implement

# Solution
orthogonal_T = is_orthogonal(T)
assert isinstance(orthogonal_T, bool)
result_string = "orthogonal" if orthogonal_T else "not orthogonal"
print(f"The scores vectors are {result_string}")

orthonormal_T = is_orthonormal(T)
assert isinstance(orthonormal_T, bool)
result_string = "orthonormal" if orthonormal_T else "not orthonormal"
print(f"The scores vectors are {result_string}")## Task 3: Add more columns to the original data matrix by: 
* Make some of the columns to be the linear combination of others
* Duplicate some columns
* Add noise as some columns 
* Add a few columns of categorical values

Then apply PCA to the dataset and report your findings here:

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler # Choose one
import matplotlib.pyplot as plt # Maybe use to visualize something


# TODO: Implement